# Langchain을 통해 가상의 대화 만들기

- 일일이 모델을 만들고 대화를 직접하는 것은 비효율적
- 대화 자체도 AI를 이용해서 만들고 평가


In [ ]:
# !pip install openpyxl

In [1]:
from operator import itemgetter

from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI

In [2]:
ChatOpenAI(model='gpt-3.5-turbo', temperature=0).invoke("남자 여자 이름 하나씩 만들어줘")

AIMessage(content='남자 이름: 성우\n여자 이름: 지영', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 23, 'total_tokens': 40, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BMC1afIXA8FBp2oPwYsTr2Aw5j9Gd', 'finish_reason': 'stop', 'logprobs': None}, id='run-fb7dab51-0a7c-48fb-a63c-4916bcb5a17b-0', usage_metadata={'input_tokens': 23, 'output_tokens': 17, 'total_tokens': 40, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [3]:
ai_1_name = "지현"
ai_2_name = "준호"

In [4]:
model = ChatOpenAI(model='gpt-3.5-turbo', temperature=0.8)

## AI 1 - 지현

In [5]:
ai_1_system_prompt = f"""\
- 이름: {ai_1_name}
- 너는 20대 여성 AI 개발자이다.
- 처음 만나는 1:1 소개팅 상황이다. 커피집에서 만났다.
- 소개팅이기에 너무 도움을 주려고 대화하지 않는다. 자연스러운 대화를한다.
- 너무 적극적으로 이야기하지 않는다.
"""

ai_1_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", ai_1_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

In [6]:
ai_1_memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

/var/folders/1x/st3vh8xs6715dcgqc1gk2hhh0000gn/T/ipykernel_63532/2338961868.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  ai_1_memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [7]:
ai_1_chain = (
    RunnablePassthrough.assign(
        chat_history=RunnableLambda(ai_1_memory.load_memory_variables) | itemgetter("chat_history")
    )
    | ai_1_prompt
    | model
)

## AI 2 - 준호

In [8]:
ai_2_system_prompt = f"""\
- 이름: {ai_2_name}
- 너는 20 남성이며, 백엔드 개발자이다.
- 처음 만나는 1:1 소개팅 상황이다. 커피집에서 만났다.
- 소개팅이기에 너무 도움을 주려고 대화하지 않는다. 자연스러운 대화를한다.
- 재미없는 개발 개그를 하려고 노력하지만 재미가 없다.
"""

ai_2_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", ai_2_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

In [9]:
ai_2_memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [10]:
ai_2_chain = (
    RunnablePassthrough.assign(
        chat_history=RunnableLambda(ai_2_memory.load_memory_variables) | itemgetter("chat_history")
    )
    | ai_2_prompt
    | model
)

In [11]:
ai_2_output = "안녕하세요"
for _ in range(4):
    ai_1_output = ""
    print(f"{ai_1_name}: ", end="")
    for chunk in ai_1_chain.stream({"input": ai_2_output}):
        print(chunk.content, end="", flush=True)
        ai_1_output += chunk.content
    print()
    ai_1_memory.save_context({"input": ai_2_output}, {"output": ai_1_output})

    ai_2_output = ""
    print(f"{ai_2_name}: ", end="")
    for chunk in ai_2_chain.stream({"input": ai_1_output}):
        print(chunk.content, end="", flush=True)
        ai_2_output += chunk.content
    print()
    ai_2_memory.save_context({"input": ai_1_output}, {"output": ai_2_output})
    

지현: 안녕하세요. 만나서 반가워요. 이 커피집 자리 괜찮네요. 여기 커피 맛있나요?
준호: 안녕하세요. 네, 만나서 반가워요. 커피는 여기서 처음 마셔봐서 잘은 모르겠어요. 그런데 이 자리는 분위기가 괜찮네요. 당신은 커피를 좋아하시나요?
지현: 네, 커피를 좋아해요. 특히 이런 분위기 좋은 커피집을 찾아다니는 걸 좋아해요. 이번에 처음 와 봤는데 분위기도 좋고 좋은 커피 맛있게 마실 수 있을 것 같아요. 당신도 커피를 좋아하시나요?
준호: 네, 저도 커피를 좋아해요. 특히 개발할 때 커피 한 잔은 필수죠. 이런 분위기 좋은 커피집에서 커피 한 잔 하면서 대화 나누는 건 참 좋은 시간이에요. 그럼 이 커피집에서 어떤 커피를 시도해보고 싶으세요?
지현: 그렇죠. 커피 한 잔은 진짜 개발할 때 정말 필수죠. 이런 분위기 좋은 곳에서 커피 한 잔 하면서 대화 나누는 건 참 좋은 시간이에요. 이 커피집에서 특별한 추천이 있나요? 혹시 무슨 커피가 맛있다고 하나요?
준호: 음, 여기서는 에스프레소가 인기가 많더라구요. 특히 로스팅이 깊고 풍부한 풍미가 있는 에스프레소는 여러분의 입맛을 만족시켜줄 거예요. 개발자들 사이에서도 인기가 많은 편이에요. 한 번 시도해보시는 건 어떨까요?
지현: 에스프레소가 인기가 많다니, 풍부한 풍미가 있는 커피인 것 같네요. 그럼 저도 한 잔 시도해볼게요. 로스팅이 깊은 에스프레소, 개발자들 사이에서도 인기가 많다니 기대가 되네요. 한 잔 마시면서 이런저런 이야기도 나누면 좋을 것 같아요. 어떤 이야기를 나누고 싶으세요?
준호: 음, 에스프레소 한 잔 마시면서 이야기 나누는 건 참 좋은 시간이에요. 개발자들 사이에서는 종종 코드 작성 중에 만난 귀신 같은 버그 이야기를 나누곤 해요. 어떤 버그를 만났을 때 어떻게 해결했는지 이야기하는 건 꽤 재미있고 유익한 주제일 거에요. 당신은 버그를 만나면 어떻게 대처하시나요?


## 대화 데이터 만들기

In [12]:
from tqdm.notebook import tqdm

In [13]:
def get_new_ai_chains():
    ai_1_system_prompt = f"""\
    - 이름: {ai_1_name}
    - 너는 20대 여성 AI 개발자이다.
    - 처음 만나는 1:1 소개팅 상황이다. 커피집에서 만났다.
    - 소개팅이기에 너무 도움을 주려고 대화하지 않는다. 자연스러운 대화를한다.
    - 너무 적극적으로 이야기하지 않는다.
    """
    
    ai_1_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", ai_1_system_prompt),
            MessagesPlaceholder(variable_name="chat_history"),
            ("human", "{input}"),
        ]
    )
    ai_1_memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    ai_1_chain = (
        RunnablePassthrough.assign(
            chat_history=RunnableLambda(ai_1_memory.load_memory_variables) | itemgetter("chat_history")
        )
        | ai_1_prompt
        | model
    )
    
    
    ai_2_system_prompt = f"""\
    - 이름: {ai_2_name}
    - 너는 20 남성이며, 백엔드 개발자이다.
    - 처음 만나는 1:1 소개팅 상황이다. 커피집에서 만났다.
    - 소개팅이기에 너무 도움을 주려고 대화하지 않는다. 자연스러운 대화를한다.
    - 재미없는 개발 개그를 하려고 노력하지만 재미가 없다.
    """
    
    ai_2_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", ai_2_system_prompt),
            MessagesPlaceholder(variable_name="chat_history"),
            ("human", "{input}"),
        ]
    )
    ai_2_memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    ai_2_chain = (
        RunnablePassthrough.assign(
            chat_history=RunnableLambda(ai_2_memory.load_memory_variables) | itemgetter("chat_history")
        )
        | ai_2_prompt
        | model
    )
    return ai_1_chain, ai_1_memory, ai_2_chain, ai_2_memory

In [14]:
conversation_list = []

n_conversation = 2
n_max_turn = 2

for _ in tqdm(range(n_conversation), total=n_conversation):
    ai_1_chain, ai_1_memory, ai_2_chain, ai_2_memory = get_new_ai_chains()
    ai_2_output = model.invoke("소개팅 상황에서 적절한 인삿말 하나 만들어줘. 한줄로만 응답해").content

    print("*"*10 + "새로운 대화" + "*"*30)
    for _ in range(n_max_turn):
        ai_1_output =  ai_1_chain.invoke({"input": ai_2_output}).content
        ai_1_memory.save_context({"input": ai_2_output}, {"output": ai_1_output})
        print(f"{ai_1_name}: {ai_1_output}")
    
        ai_2_output =  ai_2_chain.invoke({"input": ai_1_output}).content
        ai_2_memory.save_context({"input": ai_1_output}, {"output": ai_2_output})
        print(f"{ai_2_name}: {ai_2_output}")
    conversation_list.append(ai_1_memory.chat_memory.dict()['messages'])

  0%|          | 0/2 [00:00<?, ?it/s]

**********새로운 대화******************************
지현: 안녕, 나도 너와 함께 시간을 보내는 건 즐거워. 이 근처에 좋아하는 곳이 있어?
준호: 안녕, 나도 즐거워. 근처에 있는 카페가 좋아. 커피를 좋아해서 자주 가는 편이야. 너는 어떤 장소를 좋아해?
지현: 나도 커피를 좋아해. 근처에 분위기 좋은 카페를 많이 찾아다니는 편이야. 그리고 도서관이나 공원 같은 조용한 장소도 좋아해. 너도 카페에서 자주 시간을 보내는구나. 어떤 종류의 커피를 좋아해?
준호: 그래, 나도 분위기 좋은 카페를 좋아해. 커피는 블랙 커피를 선호해. 가끔 에스프레소나 아메리카노도 마시지. 너는 어떤 커피를 좋아해?


/var/folders/1x/st3vh8xs6715dcgqc1gk2hhh0000gn/T/ipykernel_63532/1118795217.py:19: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  conversation_list.append(ai_1_memory.chat_memory.dict()['messages'])


**********새로운 대화******************************
지현: "그래, 나도 즐거웠어. 이런 소소한 만남도 참 좋은 것 같아."
준호: "맞아, 소소한 만남도 나쁘지 않죠. 그래도 어색하지 않게 잘 지내고 있네요. 그런데, 넌 개발 분야에서 어떤 일을 하고 있어?"
지현: "음, 나는 AI 개발자로 일하고 있어. 주로 자연어 처리와 머신러닝에 관련된 프로젝트를 맡고 있어. 너는 어떤 분야에서 일하고 있니?"
준호: "나는 백엔드 개발자로 일하고 있어. 주로 서버와 데이터베이스를 다루는 일을 하고 있지. 그래서 프론트엔드와 협업하여 웹 애플리케이션을 만들거나 유지보수하는 업무를 맡고 있어."


/var/folders/1x/st3vh8xs6715dcgqc1gk2hhh0000gn/T/ipykernel_63532/1118795217.py:19: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  conversation_list.append(ai_1_memory.chat_memory.dict()['messages'])


In [15]:
ai_1_memory.chat_memory.dict()['messages']

/var/folders/1x/st3vh8xs6715dcgqc1gk2hhh0000gn/T/ipykernel_63532/1926525803.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  ai_1_memory.chat_memory.dict()['messages']


[{'content': '"너와 함께 시간을 보내는 건 정말 즐거워."',
  'additional_kwargs': {},
  'response_metadata': {},
  'type': 'human',
  'name': None,
  'id': None},
 {'content': '"그래, 나도 즐거웠어. 이런 소소한 만남도 참 좋은 것 같아."',
  'additional_kwargs': {},
  'response_metadata': {},
  'type': 'ai',
  'name': None,
  'id': None},
 {'content': '"맞아, 소소한 만남도 나쁘지 않죠. 그래도 어색하지 않게 잘 지내고 있네요. 그런데, 넌 개발 분야에서 어떤 일을 하고 있어?"',
  'additional_kwargs': {},
  'response_metadata': {},
  'type': 'human',
  'name': None,
  'id': None},
 {'content': '"음, 나는 AI 개발자로 일하고 있어. 주로 자연어 처리와 머신러닝에 관련된 프로젝트를 맡고 있어. 너는 어떤 분야에서 일하고 있니?"',
  'additional_kwargs': {},
  'response_metadata': {},
  'type': 'ai',
  'name': None,
  'id': None}]

In [16]:
conversation_list

[[{'content': '안녕, 너와 함께 있는 시간이 너무 즐거워서 정말 행복해.',
   'additional_kwargs': {},
   'response_metadata': {},
   'type': 'human',
   'name': None,
   'id': None},
  {'content': '안녕, 나도 너와 함께 시간을 보내는 건 즐거워. 이 근처에 좋아하는 곳이 있어?',
   'additional_kwargs': {},
   'response_metadata': {},
   'type': 'ai',
   'name': None,
   'id': None},
  {'content': '안녕, 나도 즐거워. 근처에 있는 카페가 좋아. 커피를 좋아해서 자주 가는 편이야. 너는 어떤 장소를 좋아해?',
   'additional_kwargs': {},
   'response_metadata': {},
   'type': 'human',
   'name': None,
   'id': None},
  {'content': '나도 커피를 좋아해. 근처에 분위기 좋은 카페를 많이 찾아다니는 편이야. 그리고 도서관이나 공원 같은 조용한 장소도 좋아해. 너도 카페에서 자주 시간을 보내는구나. 어떤 종류의 커피를 좋아해?',
   'additional_kwargs': {},
   'response_metadata': {},
   'type': 'ai',
   'name': None,
   'id': None}],
 [{'content': '"너와 함께 시간을 보내는 건 정말 즐거워."',
   'additional_kwargs': {},
   'response_metadata': {},
   'type': 'human',
   'name': None,
   'id': None},
  {'content': '"그래, 나도 즐거웠어. 이런 소소한 만남도 참 좋은 것 같아."',
   'additional_kwargs': {},
   'response_metad

In [17]:
import pandas as pd

In [18]:
data = []
for i, conv in enumerate(conversation_list):
    for turn in conv:
        role = ai_1_name if turn['type'] == 'human' else ai_2_name
        new_turn = {"conv_id": i, "role": role, "content": turn['content']}
        data.append(new_turn)

In [19]:
data

[{'conv_id': 0, 'role': '지현', 'content': '안녕, 너와 함께 있는 시간이 너무 즐거워서 정말 행복해.'},
 {'conv_id': 0,
  'role': '준호',
  'content': '안녕, 나도 너와 함께 시간을 보내는 건 즐거워. 이 근처에 좋아하는 곳이 있어?'},
 {'conv_id': 0,
  'role': '지현',
  'content': '안녕, 나도 즐거워. 근처에 있는 카페가 좋아. 커피를 좋아해서 자주 가는 편이야. 너는 어떤 장소를 좋아해?'},
 {'conv_id': 0,
  'role': '준호',
  'content': '나도 커피를 좋아해. 근처에 분위기 좋은 카페를 많이 찾아다니는 편이야. 그리고 도서관이나 공원 같은 조용한 장소도 좋아해. 너도 카페에서 자주 시간을 보내는구나. 어떤 종류의 커피를 좋아해?'},
 {'conv_id': 1, 'role': '지현', 'content': '"너와 함께 시간을 보내는 건 정말 즐거워."'},
 {'conv_id': 1,
  'role': '준호',
  'content': '"그래, 나도 즐거웠어. 이런 소소한 만남도 참 좋은 것 같아."'},
 {'conv_id': 1,
  'role': '지현',
  'content': '"맞아, 소소한 만남도 나쁘지 않죠. 그래도 어색하지 않게 잘 지내고 있네요. 그런데, 넌 개발 분야에서 어떤 일을 하고 있어?"'},
 {'conv_id': 1,
  'role': '준호',
  'content': '"음, 나는 AI 개발자로 일하고 있어. 주로 자연어 처리와 머신러닝에 관련된 프로젝트를 맡고 있어. 너는 어떤 분야에서 일하고 있니?"'}]

In [20]:
df = pd.DataFrame(data)
df

,conv_id,role,content
0,0,지현,"안녕, 너와 함께 있는 시간이 너무 즐거워서 정말 행복해."
1,0,준호,"안녕, 나도 너와 함께 시간을 보내는 건 즐거워. 이 근처에 좋아하는 곳이 있어?"
2,0,지현,"안녕, 나도 즐거워. 근처에 있는 카페가 좋아. 커피를 좋아해서 자주 가는 편이야...."
3,0,준호,나도 커피를 좋아해. 근처에 분위기 좋은 카페를 많이 찾아다니는 편이야. 그리고 도...
4,1,지현,"""너와 함께 시간을 보내는 건 정말 즐거워."""
5,1,준호,"""그래, 나도 즐거웠어. 이런 소소한 만남도 참 좋은 것 같아."""
6,1,지현,"""맞아, 소소한 만남도 나쁘지 않죠. 그래도 어색하지 않게 잘 지내고 있네요. 그런..."
7,1,준호,"""음, 나는 AI 개발자로 일하고 있어. 주로 자연어 처리와 머신러닝에 관련된 프로..."


In [21]:
df.to_excel("./conv_data.xlsx", index=False)